In [1]:
!pip install pyforest
import pyforest


  Preparing metadata (setup.py) ... done
  Created wheel for pyforest: filename=pyforest-1.1.0-py2.py3-none-any.whl size=14605 sha256=29440be0da6f777d746ad67d65b29ae6104f5f207bad70be2862db1a3a44df14
  Stored in directory: /root/.cache/pip/wheels/9e/7d/2c/5d2f5e62de376c386fd3bf5a8e5bd119ace6a9f48f49df6017
Successfully built pyforest


In [2]:
import os

In [3]:
path = '/content/drive/MyDrive/CodSoft/Data'

In [4]:
train = os.path.join(path, 'train_data.txt')
test = os.path.join(path, 'test_data.txt')

In [5]:
train_df = pd.read_csv(train, sep=':::', header=None, engine='python')
test_df = pd.read_csv(test, sep=':::', header=None, engine='python')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [6]:
train_df.columns = ['Id', 'Title', 'Genre', 'Description']
test_df.columns = ['Id', 'Title', 'Description']

In [7]:
train_df.head()

,Id,Title,Genre,Description
0,1,Oscar et la dame rose (2009),drama,Listening in to a conversation between his do...
1,2,Cupid (1997),thriller,A brother and sister with a past incestuous r...
2,3,"Young, Wild and Wonderful (1980)",adult,As the bus empties the students for their fie...
3,4,The Secret Sin (1915),drama,To help their unemployed father make ends mee...
4,5,The Unrecovered (2007),drama,The film's title refers not only to the un-re...


In [8]:
train_df.drop(['Id', 'Title'], inplace=True, axis=1)

In [9]:
train_df.head()

,Genre,Description
0,drama,Listening in to a conversation between his do...
1,thriller,A brother and sister with a past incestuous r...
2,adult,As the bus empties the students for their fie...
3,drama,To help their unemployed father make ends mee...
4,drama,The film's title refers not only to the un-re...


In [10]:
## LowerCasing - Reduce vocabulary size and promote comsistency.

In [11]:
train_df['Description'] = train_df['Description'].str.lower()

In [12]:
train_df.Description[0]

' listening in to a conversation between his doctor and parents, 10-year-old oscar learns what nobody has the courage to tell him. he only has a few weeks to live. furious, he refuses to speak to anyone except straight-talking rose, the lady in pink he meets on the hospital stairs. as christmas approaches, rose uses her fantastical experiences as a professional wrestler, her imagination, wit and charm to allow oscar to live life and love to the full, in the company of his friends pop corn, einstein, bacon and childhood sweetheart peggy blue.'

In [13]:
!pip install nltk

In [14]:
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer
import string


stemmer = LancasterStemmer()
stop_words = set(stopwords.words('english'))

# Define the clean_text function
def clean_text(text):
    text = text.lower()  # Lowercase all characters
    text = re.sub(r'@\S+', '', text)  # Remove Twitter handles
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'pic.\S+', '', text)
    text = re.sub(r"[^a-zA-Z+']", ' ', text)  # Keep only characters
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text + ' ')  # Keep words with length > 1 only
    text = "".join([i for i in text if i not in string.punctuation])
    words = nltk.word_tokenize(text)
    stopwords = nltk.corpus.stopwords.words('english')  # Remove stopwords
    text = " ".join([i for i in words if i not in stopwords and len(i) > 2])
    text = re.sub("\s[\s]+", " ", text).strip()  # Remove repeated/leading/trailing spaces
    return text

# Apply the clean_text function to the 'Description' column in the training and test data
train_df['Text_cleaning'] = train_df['Description'].apply(clean_text)
test_df['Text_cleaning'] = test_df['Description'].apply(clean_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [15]:

train_df.head()

,Genre,Description,Text_cleaning
0,drama,listening in to a conversation between his do...,listening conversation doctor parents year old...
1,thriller,a brother and sister with a past incestuous r...,brother sister past incestuous relationship cu...
2,adult,as the bus empties the students for their fie...,bus empties students field trip museum natural...
3,drama,to help their unemployed father make ends mee...,help unemployed father make ends meet edith tw...
4,drama,the film's title refers not only to the un-re...,films title refers recovered bodies ground zer...


In [16]:
def apply_lancaster_stemming(text):
    tokens = text.split()
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    return " ".join(stemmed_tokens)

train_df['Description_Cleaned'] = train_df['Text_cleaning'].apply(apply_lancaster_stemming)
test_df['Description_Cleaned'] = test_df['Text_cleaning'].apply(apply_lancaster_stemming)

In [17]:
train_df.head()

,Genre,Description,Text_cleaning,Description_Cleaned
0,drama,listening in to a conversation between his do...,listening conversation doctor parents year old...,list convers doct par year old osc learn nobod...
1,thriller,a brother and sister with a past incestuous r...,brother sister past incestuous relationship cu...,broth sist past incestu rel cur murd rel murd ...
2,adult,as the bus empties the students for their fie...,bus empties students field trip museum natural...,bus empty stud field trip muse nat hist littl ...
3,drama,to help their unemployed father make ends mee...,help unemployed father make ends meet edith tw...,help unemploy fath mak end meet edi twin sist ...
4,drama,the film's title refers not only to the un-re...,films title refers recovered bodies ground zer...,film titl ref recov body ground zero also stat...


In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X = tfidf_vectorizer.fit_transform(train_df["Description_Cleaned"]).toarray()
y = train_df["Genre"]

In [19]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)


In [20]:
y[1: 6]

array([24,  1,  8,  8,  7])

In [21]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [22]:
label_encoder.classes_

array([' action ', ' adult ', ' adventure ', ' animation ', ' biography ',
       ' comedy ', ' crime ', ' documentary ', ' drama ', ' family ',
       ' fantasy ', ' game-show ', ' history ', ' horror ', ' music ',
       ' musical ', ' mystery ', ' news ', ' reality-tv ', ' romance ',
       ' sci-fi ', ' short ', ' sport ', ' talk-show ', ' thriller ',
       ' war ', ' western '], dtype=object)

In [23]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding, Dropout
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score




input_dim = X_train.shape[1]  # Dimension of input features (TF-IDF vectorizer outputs)
num_classes = len(label_encoder.classes_)  # Number of output classes (movie genres)

model = Sequential([
    Embedding(input_dim=input_dim, output_dim=64, input_length=input_dim),
    LSTM(64),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=15, batch_size=32, validation_split=0.1)

# Step 5: Model Evaluation



Epoch 1/15
1220/1220 [==============================] - 49s 35ms/step - loss: 2.3822 - accuracy: 0.2452 - val_loss: 2.3698 - val_accuracy: 0.2453
Epoch 2/15
1220/1220 [==============================] - 42s 34ms/step - loss: 2.3532 - accuracy: 0.2472 - val_loss: 2.3431 - val_accuracy: 0.2453
Epoch 3/15
1220/1220 [==============================] - 41s 33ms/step - loss: 2.3486 - accuracy: 0.2458 - val_loss: 2.3329 - val_accuracy: 0.2609
Epoch 4/15
1220/1220 [==============================] - 41s 34ms/step - loss: 2.3451 - accuracy: 0.2487 - val_loss: 2.3330 - val_accuracy: 0.2453
Epoch 5/15
1220/1220 [==============================] - 41s 34ms/step - loss: 2.3446 - accuracy: 0.2468 - val_loss: 2.3312 - val_accuracy: 0.2609
Epoch 6/15
1220/1220 [==============================] - 41s 34ms/step - loss: 2.3437 - accuracy: 0.2464 - val_loss: 2.3332 - val_accuracy: 0.2453
Epoch 7/15
1220/1220 [==============================] - 42s 34ms/step - loss: 2.3432 - accuracy: 0.2460 - val_loss: 2.3319 -

In [24]:
y_pred_prob = model.predict(X_test)
y_pred = np.argmax(y_pred_prob, axis=1)

339/339 [==============================] - 5s 14ms/step


<IPython.core.display.Javascript object>

In [25]:
y_pred[1]

8

In [26]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.25


In [27]:
y_test_original = label_encoder.inverse_transform(y_test)
y_pred_original = label_encoder.inverse_transform(y_pred)
print(classification_report(y_test_original, y_pred_original))

               precision    recall  f1-score   support

      action        0.00      0.00      0.00       263
       adult        0.00      0.00      0.00       112
   adventure        0.00      0.00      0.00       139
   animation        0.00      0.00      0.00       104
   biography        0.00      0.00      0.00        61
      comedy        0.00      0.00      0.00      1443
       crime        0.00      0.00      0.00       107
 documentary        0.00      0.00      0.00      2659
       drama        0.25      1.00      0.40      2697
      family        0.00      0.00      0.00       150
     fantasy        0.00      0.00      0.00        74
   game-show        0.00      0.00      0.00        40
     history        0.00      0.00      0.00        45
      horror        0.00      0.00      0.00       431
       music        0.00      0.00      0.00       144
     musical        0.00      0.00      0.00        50
     mystery        0.00      0.00      0.00        56
        n

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [28]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators=100, random_state=42)
rfc.fit(X_train, y_train)

y_pred = rfc.predict(X_test)

In [29]:
X_train.shape

(43371, 1000)

In [30]:
y_train.shape

(43371,)

In [31]:
y_pred[1]

8

In [32]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.50      0.01      0.01       263
           1       0.88      0.06      0.12       112
           2       0.44      0.05      0.09       139
           3       0.00      0.00      0.00       104
           4       0.00      0.00      0.00        61
           5       0.44      0.34      0.38      1443
           6       1.00      0.01      0.02       107
           7       0.56      0.85      0.68      2659
           8       0.43      0.81      0.56      2697
           9       1.00      0.03      0.05       150
          10       0.00      0.00      0.00        74
          11       0.89      0.40      0.55        40
          12       0.00      0.00      0.00        45
          13       0.60      0.24      0.34       431
          14       0.65      0.18      0.28       144
          15       0.00      0.00      0.00        50
          16       0.00      0.00      0.00        56
          17       0.00    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [33]:
## CNN classifier

In [36]:
cnn_data = train_df[['Genre', 'Description']]

In [37]:
cnn_data.head()

,Genre,Description
0,drama,listening in to a conversation between his do...
1,thriller,a brother and sister with a past incestuous r...
2,adult,as the bus empties the students for their fie...
3,drama,to help their unemployed father make ends mee...
4,drama,the film's title refers not only to the un-re...


In [38]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, Dense

In [39]:
cnn_data_tokenizer = Tokenizer()
cnn_data_tokenizer.fit_on_texts(cnn_data['Description'])
sequence = cnn_data_tokenizer.texts_to_sequences(cnn_data['Description'])

#pad sequences
max_length = 100
cnn_data_padded = pad_sequences(sequence, maxlen=max_length)

In [50]:
#scale cnn_data_padded

cnn_data_padded = cnn_data_padded / cnn_data_padded.max()
cnn_data_padded[0]

array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 3.78310487e-02, 4.33097295e-05, 3.60914413e-05,
       2.16548648e-05, 1.45809423e-02, 6.92955672e-04, 5.77463060e-05,
       3.79681962e-03, 1.44365765e-05, 1.66742459e-03, 4.67745079e-03,
       6.56864231e-04, 4.47533872e-04, 1.90129713e-02, 3.17604683e-03,
       3.97005854e-04, 1.66092813e-02, 1.87675495e-04, 7.21828826e-06,
       1.05603557e-02, 3.60914413e-05, 3.29875773e-03, 2.30985224e-04,
       7.94011708e-05, 4.25879007e-04, 1.87675495e-04, 2.16548648e-05,
       2.10774017e-03, 9.13835293e-03, 3.60914413e-05, 9.16722608e-04,
       3.20708547e-02, 7.94011708e-05, 6.81406411e-03, 3.60914413e-05,
       1.06108837e-02, 3.60914413e-05, 6.27269249e-03, 1.35992551e-02,
       1.16214441e-02, 1.05675740e-02, 1.04087717e-02, 7.21828826e-06,
       5.78184889e-03, 4.33097295e-05, 3.67483055e-02, 7.94011708e-05,
       1.39312963e-03, 1.01056036e-04, 7.21828826e-06, 4.09998773e-03,
      

In [51]:
#encode labels
label_encoder = LabelEncoder()
cnn_data['Genre'] = label_encoder.fit_transform(cnn_data['Genre'])

<ipython-input-51-c7dabde460a5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cnn_data['Genre'] = label_encoder.fit_transform(cnn_data['Genre'])


In [40]:
num_classes = len(cnn_data['Genre'].unique())
print(num_classes)

27


In [41]:
vocab_size = len(cnn_data_tokenizer.word_index) + 1

In [42]:
vocab_size

138538

In [45]:
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=100, input_length=max_length),
    Conv1D(128, 5, activation='relu'),
    MaxPooling1D(2),
    Conv1D(128, 5, activation='relu'),
    MaxPooling1D(2),
    Conv1D(128, 5, activation='relu'),
    MaxPooling1D(2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(num_classes, activation='softmax')
])

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 100, 100)          13853800  
                                                                 
 conv1d_3 (Conv1D)           (None, 96, 128)           64128     
                                                                 
 max_pooling1d_3 (MaxPoolin  (None, 48, 128)           0         
 g1D)                                                            
                                                                 
 conv1d_4 (Conv1D)           (None, 44, 128)           82048     
                                                                 
 max_pooling1d_4 (MaxPoolin  (None, 22, 128)           0         
 g1D)                                                            
                                                                 
 conv1d_5 (Conv1D)           (None, 18, 128)          

In [46]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [52]:
model.fit(cnn_data_padded, cnn_data['Genre'], epochs=10, batch_size=32, validation_split=0.1)

Epoch 1/10
1525/1525 [==============================] - 19s 9ms/step - loss: 2.3606 - accuracy: 0.2486 - val_loss: 2.3596 - val_accuracy: 0.2372
Epoch 2/10
1525/1525 [==============================] - 14s 9ms/step - loss: 2.3468 - accuracy: 0.2417 - val_loss: 2.3553 - val_accuracy: 0.2527
Epoch 3/10
1525/1525 [==============================] - 14s 9ms/step - loss: 2.3453 - accuracy: 0.2473 - val_loss: 2.3529 - val_accuracy: 0.2372
Epoch 4/10
1525/1525 [==============================] - 14s 9ms/step - loss: 2.3446 - accuracy: 0.2485 - val_loss: 2.3527 - val_accuracy: 0.2527
Epoch 5/10
1525/1525 [==============================] - 14s 9ms/step - loss: 2.3440 - accuracy: 0.2478 - val_loss: 2.3504 - val_accuracy: 0.2527
Epoch 6/10
1525/1525 [==============================] - 14s 9ms/step - loss: 2.3434 - accuracy: 0.2459 - val_loss: 2.3500 - val_accuracy: 0.2372
Epoch 7/10
1525/1525 [==============================] - 14s 9ms/step - loss: 2.3433 - accuracy: 0.2480 - val_loss: 2.3503 - val_ac